In [1]:
from spark_batch.lib.spark_session import get_spark_session
from spark_batch.lib.elt_manager import EltManager
from spark_batch.lib.pxlogger import CustomLogger
from spark_batch.lib.util import toTargetCondition
import logging
import os 

CustomLogger.set_custom_log_level(logging.DEBUG)  #default log_level is 'INFO'
spark = get_spark_session(app_name="f_load_subway_info",notebook_name="test", notebook_namespace="demo01-kf", \
            executor_instances="1", executor_memory="1G", executor_cores="1", driver_memory="1G", driver_cores="1" )
em = EltManager(spark, config_file="config.yaml", domain="life", record_type="mart", record_prefix="hopt")

:: loading settings :: url = jar:file:/usr/local/spark-3.5.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk added as a dependency
io.delta#delta-spark_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fba1c6ad-f9e1-4191-8dc0-1a7a8e04bf38;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-common;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-protobuf_3_7;1.1.1 in central
	found org.apache.hadoop#hadoop-annotations;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-guava;1.1.1 in central
	found com.google.guava#guava;27.0-jre in central
	found com.google.guava#failureaccess;1.0 in central
	found com.google.guava#listenablefuture;9999.0-empty-to-avoid-conflict-with-guava in central
	found com.google.code.findbugs#jsr305;3.0.2 i

2025-07-01 02:47:11,239 - DEBUG - ResourceManager:021 - ('config keys=', dict_keys(['postgresql-life', 'csv1', 'json1', 'delta1', 'mart']))


### JSON > Bronze

In [2]:
source_type = "csv1";  source_topic = "paasup"; source_dpath="datasource/life"
target_type = "delta1";   target_topic = "paasup"; target_dpath="bronze/life"

em.init_rsm(source_type, source_topic, source_dpath, target_type, target_topic, target_dpath, 500000)

2025-07-01 02:47:11,247 - DEBUG - ResourceManager:047 - s3_ssl_verify is False
2025-07-01 02:47:11,951 - DEBUG - ResourceManager:052 - ('source_type_name = delata  :', 'paasup', 'bronze/life')


In [3]:
df = spark.read.option("header","true").csv("s3a://paasup/datasource/life/subway_info.csv")
df.printSchema()

25/07/01 02:47:13 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/07/01 02:47:13 WARN AmazonHttpClient: SSL Certificate checking for endpoints has been explicitly disabled.


root
 |-- 전철역코드: string (nullable = true)
 |-- 전철역명: string (nullable = true)
 |-- 전철명명_영문: string (nullable = true)
 |-- 호선: string (nullable = true)
 |-- 외부코드: string (nullable = true)
 |-- 전철명명_중문: string (nullable = true)
 |-- 전철명명_일문: string (nullable = true)



In [4]:
df = spark.read \
                    .option("header", True) \
                    .option("delimiter", "," ) \
                    .csv("s3a://paasup/datasource/life/subway_info.csv")

In [5]:
df.createOrReplaceTempView("tmp_subway_time")
df.show()

+----------+--------+--------------+------+--------+-------------+--------------+
|전철역코드|전철역명| 전철명명_영문|  호선|외부코드|전철명명_중문| 전철명명_일문|
+----------+--------+--------------+------+--------+-------------+--------------+
|      1724|    평택|    Pyeongtaek|01호선|    P165|          平?|    ピョンテク|
|      1701|    구로|          Guro|01호선|     141|         九老|          クロ|
|      1002|    남영|      Namyeong|01호선|     134|          南?|      ナミョン|
|      1706|    안양|        Anyang|01호선|    P147|          安?|      アニヤン|
|       159|  동묘앞|       Dongmyo|01호선|     127|           ??|  トンミョアプ|
|      1720|    진위|         Jinwi|01호선|    P161|         振威|        チヌィ|
|      1804|    부천|       Bucheon|01호선|     148|         富川|      プチョン|
|      1707|    명학|     Myeonghak|01호선|    P148|           ??|    ミョンハク|
|      1015|    회기|         Hoegi|01호선|     123|         回基|        フェギ|
|      1719|    오산|          Osan|01호선|    P160|          ?山|        オサン|
|      1404|    탕정|     Tangjeong|01호선|    P173|          ?

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException

def s3_file_exist(spark, s3_path):
    """
    S3에 특정 파일이나 디렉터리가 존재하는지 확인하고 결과를 boolean으로 반환합니다.

    :param spark: 활성화된 SparkSession 객체.
    :param s3_path: 확인할 S3 파일 또는 디렉터리의 전체 경로 (e.g., "s3a://bucket/path/file.txt").
    :return: 파일 또는 디렉터리가 존재하면 True, 그렇지 않으면 False를 반환.
    """
    try:
        # spark.read.load()는 경로가 존재하지 않으면 AnalysisException을 발생시킵니다.
        # 실제 데이터를 읽지는 않고 경로의 유효성만 확인하는 효과를 얻습니다.
        #spark.read.load(s3_path)  #파일의 인코딩이나 메타정보 읽다가 실패할 수도 있음 
        spark.read.text(s3_path).limit(0).count()
        return True
    except AnalysisException as e:
        # 경로가 존재하지 않을 때 발생하는 특정 예외를 확인합니다.
        if 'Path does not exist' in str(e):
            return False
        else:
            # 그 외 다른 분석 예외 (예: 권한 문제)는 오류로 출력합니다.
            print(f"파일 확인 중 분석 오류가 발생했습니다: {e}")
            return False
    except Exception as e:
        # 그 외 예상치 못한 예외 (네트워크 문제 등)
        print(f"파일 확인 중 예상치 못한 오류가 발생했습니다: {e}")
        return False

print(s3_file_exist(spark, "s3a://paasup/datasource/life/서울교통공사_노선별 지하철역 정보.csv"))


False


In [7]:
source_objects = ["subway_info.csv"]; target_object = "SUBWAY_INFO"
#source_tm = em.getSourceManager()
#source_df = source_tm.loadTable(source_objects[0])

#source_inc_query = f"""
#select * from SUBWAY_INFO
#""" 

source_df, target_df, valid = em.ingest_full(source_objects, target_object, delemeter="," )

2025-07-01 02:47:57,080 - INFO - EltManager:194 - ETL/FL Started : [ delta1 paasup bronze/life SUBWAY_INFO (['subway_info.csv']) ]
2025-07-01 02:47:57,081 - DEBUG - CSVTableManager:042 - ('loadTable = ', 'subway_info.csv', None, None)
2025-07-01 02:47:57,082 - DEBUG - CSVTableManager:045 - ('file_path = ', 's3a://paasup/datasource/life/subway_info.csv')
+----------+--------+-------------+------+--------+-------------+-------------+
|전철역코드|전철역명|전철명명_영문|  호선|외부코드|전철명명_중문|전철명명_일문|
+----------+--------+-------------+------+--------+-------------+-------------+
|      1724|    평택|   Pyeongtaek|01호선|    P165|          平?|   ピョンテク|
|      1701|    구로|         Guro|01호선|     141|         九老|         クロ|
|      1002|    남영|     Namyeong|01호선|     134|          南?|     ナミョン|
|      1706|    안양|       Anyang|01호선|    P147|          安?|     アニヤン|
|       159|  동묘앞|      Dongmyo|01호선|     127|           ??| トンミョアプ|
+----------+--------+-------------+------+--------+-------------+-------------+
only

25/07/01 02:48:03 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/07/01 02:48:16 WARN AmazonHttpClient: SSL Certificate checking for endpoints has been explicitly disabled.


2025-07-01 02:48:21,322 - DEBUG - DeltaTableManager:119 - saveTable : before = 0, after = 796 [ SUBWAY_INFO ]
2025-07-01 02:48:26,843 - DEBUG - DeltaTableManager:338 - 
        version       operation  numDeletedRows   numOutputRows                        operationParameters
              2           WRITE            None             796 {'mode': 'Overwrite', 'partitionBy': '[]'}
2025-07-01 02:48:26,843 - INFO - EltManager:268 - Source Loading Chunk : csv1 paasup datasource/life ['subway_info.csv'] / seq=1 offset=0 chunk_size=796 / elipsed=28.25
2025-07-01 02:48:26,844 - INFO - EltManager:270 - Target Saving Chunk : delta1 paasup bronze/life SUBWAY_INFO / elipsed=0.00
2025-07-01 02:48:26,914 - DEBUG - CSVTableManager:124 - ('archive = ', 'subway_info.csv', 'paasup', 'datasource/life')
2025-07-01 02:48:26,941 - DEBUG - CSVTableManager:133 - ('archive filelists=', ['datasource/life/', 'datasource/life/CARD_SUBWAY_MONTH_202502.csv', 'datasource/life/CARD_SUBWAY_MONTH_202504.csv', 'datasou

2025-07-01 02:48:28,315 - DEBUG - DeltaTableManager:084 - ('dt_with_row size & offset = ', 796, None)


2025-07-01 02:48:29,604 - INFO - EltManager:284 - Target Saving Count : delta1 paasup bronze/life SUBWAY_INFO (796)


2025-07-01 02:48:31,826 - INFO - EltManager:288 - ETL/FL Done : [ delta1 paasup bronze/life SUBWAY_INFO / True (796, 796, 0) / 34.75 ]


2025-07-01 02:48:32,966 - DEBUG - EltManager:115 - self.record is True
2025-07-01 02:48:33,102 - DEBUG - PostgreSQLTableManager:302 - ('saveTable = ', 'elt_manager_log', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')
2025-07-01 02:48:33,103 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'elt_manager_log', None, 100000)
2025-07-01 02:48:33,103 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'public.elt_manager_log')
2025-07-01 02:48:33,104 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')


2025-07-01 02:48:35,329 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'elt_manager_log', None, 100000)
2025-07-01 02:48:35,330 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'public.elt_manager_log')
2025-07-01 02:48:35,331 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')


2025-07-01 02:48:35,521 - DEBUG - PostgreSQLTableManager:334 - saveTable : before = 9640, after = 9641 [ public.elt_manager_log ]


### Bronze > Gold

In [8]:
source_type = "delta1"; source_topic = "paasup";  source_dpath="bronze/life"
target_type = "delta1"; target_topic = "paasup";  target_dpath="gold"
em.init_rsm(source_type, source_topic, source_dpath, target_type, target_topic, target_dpath, 50000000)

2025-07-01 02:48:48,637 - DEBUG - ResourceManager:052 - ('source_type_name = delata  :', 'paasup', 'bronze/life')
2025-07-01 02:48:48,651 - DEBUG - ResourceManager:052 - ('source_type_name = delata  :', 'paasup', 'gold')


In [9]:
source_objects = ["SUBWAY_INFO"];  target_object = "subway_info"

source_inc_query = f"""
                        select  `전철역명` station_name
                           ,`전철역코드` station_cd
                           ,`전철명명_영문`  station_ename
                           ,`전철명명_중문`  station_cname
                           ,`전철명명_일문`  station_jname
                           ,`외부코드`  station_cd_external
                           , case when substring(`호선`,1,1) = '0' then substring(`호선`,2,length(`호선`)-1) 
                                  when `호선` = '공항철도' then '공항철도 1호선' 
                                  when `호선` = '수인분당선' then '수인선'
                                  when `호선` = '신분당선' then '분당선'
                                  when `호선` = '우이신설경전철' then '우이신설선' 
                                  else `호선` end   line_no
                        from SUBWAY_INFO
"""
# 전부 삭제 
target_condition = f"""
 1=1
"""
source_df, target_df, valid = em.ingest_increment(source_objects, target_object, source_inc_query, target_condition)

2025-07-01 02:48:52,691 - INFO - EltManager:334 - ETL/IC Started : [ delta1 paasup gold subway_info (['SUBWAY_INFO']) ]
2025-07-01 02:48:52,691 - DEBUG - DeltaTableManager:155 - ('queryTable = ', "\n                        select  `전철역명` station_name\n                           ,`전철역코드` station_cd\n                           ,`전철명명_영문`  station_ename\n                           ,`전철명명_중문`  station_cname\n                           ,`전철명명_일문`  station_jname\n                           ,`외부코드`  station_cd_external\n                           , case when substring(`호선`,1,1) = '0' then substring(`호선`,2,length(`호선`)-1) \n                                  when `호선` = '공항철도' then '공항철도 1호선' \n                                  when `호선` = '수인분당선' then '수인선'\n                                  when `호선` = '신분당선' then '분당선'\n                                  when `호선` = '우이신설경전철' then '우이신설선' \n                                  else `호선` end   line_no\n                        from SUBWAY_INFO\n",

25/07/01 02:49:00 WARN AmazonHttpClient: SSL Certificate checking for endpoints has been explicitly disabled.
25/07/01 02:49:04 WARN AmazonHttpClient: SSL Certificate checking for endpoints has been explicitly disabled.


2025-07-01 02:49:08,073 - DEBUG - DeltaTableManager:200 - delSert : before = 796, after = 796, del = 796 [ s3a://paasup/gold/subway_info / 
 1=1
 ]


2025-07-01 02:49:08,616 - DEBUG - DeltaTableManager:338 - 
        version       operation  numDeletedRows   numOutputRows                     operationParameters
              6           WRITE            None             796 {'mode': 'Append', 'partitionBy': '[]'}
              5          DELETE             796            None               {'predicate': '["true"]'}
2025-07-01 02:49:09,636 - INFO - EltManager:369 - Source Loading : delta1 paasup bronze/life ['SUBWAY_INFO'] / source_size=796 / elipsed=16.95
2025-07-01 02:49:09,637 - INFO - EltManager:371 - Target  Saving : delta1 paasup gold subway_info / delsert_size=796 (before=796, after=796, del=796) / elipsed=0.00


2025-07-01 02:49:10,665 - DEBUG - DeltaTableManager:284 - ('target_table=', 's3a://paasup/gold/subway_info', ', condition= ', '\n 1=1\n', ', count=', 796)
2025-07-01 02:49:10,666 - INFO - EltManager:375 - ETL/IC Done : [ delta1 paasup gold subway_info / True (796, 796, 0) / 17.98 ]
2025-07-01 02:49:10,666 - DEBUG - EltManager:115 - self.record is True
2025-07-01 02:49:10,702 - DEBUG - PostgreSQLTableManager:302 - ('saveTable = ', 'elt_manager_log', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')
2025-07-01 02:49:10,702 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'elt_manager_log', None, 100000)
2025-07-01 02:49:10,703 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'public.elt_manager_log')
2025-07-01 02:49:10,703 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')


2025-07-01 02:49:11,114 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'elt_manager_log', None, 100000)
2025-07-01 02:49:11,115 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'public.elt_manager_log')
2025-07-01 02:49:11,116 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')
2025-07-01 02:49:11,239 - DEBUG - PostgreSQLTableManager:334 - saveTable : before = 9641, after = 9642 [ public.elt_manager_log ]


### Gold > MART

In [10]:
source_type = "delta1"; source_topic = "paasup";  source_dpath="gold"
target_type = "mart"; target_topic = "postgres";  target_dpath=None
em.init_rsm(source_type, source_topic, source_dpath, target_type, target_topic, target_dpath, 50000000)

2025-07-01 02:49:40,650 - DEBUG - ResourceManager:052 - ('source_type_name = delata  :', 'paasup', 'gold')


In [11]:
source_objects = ["subway_info"];  target_object = "subway_info"
source_inc_query = f"""
                        select  *
                        from subway_info
"""
# 전부 삭제 
target_condition = f"""
 1=1
"""
source_df, target_df, valid = em.ingest_increment(source_objects, target_object, source_inc_query, target_condition)

2025-07-01 02:49:43,752 - INFO - EltManager:334 - ETL/IC Started : [ mart postgres _ subway_info (['subway_info']) ]
2025-07-01 02:49:43,753 - DEBUG - DeltaTableManager:155 - ('queryTable = ', '\n                        select  *\n                        from subway_info\n', ['subway_info'], None, None, None)
2025-07-01 02:49:43,753 - DEBUG - DeltaTableManager:127 - ('loadTables = ', ['subway_info'], None, None, None)
2025-07-01 02:49:43,826 - DEBUG - PostgreSQLTableManager:397 - ('delSert = ', '\n 1=1\n', 'subway_info')
2025-07-01 02:49:43,826 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'subway_info', None, 100000)
2025-07-01 02:49:43,827 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'postgres.public.subway_info')
2025-07-01 02:49:43,827 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')
2025-07-01 02:49:44,126 - DEBUG - PostgreSQLTable

2025-07-01 02:49:47,178 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'subway_info', None, 100000)
2025-07-01 02:49:47,179 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'postgres.public.subway_info')
2025-07-01 02:49:47,180 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')
2025-07-01 02:49:47,326 - DEBUG - PostgreSQLTableManager:334 - saveTable : before = 0, after = 796 [ postgres.public.subway_info ]
2025-07-01 02:49:47,327 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'subway_info', None, 100000)
2025-07-01 02:49:47,327 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'postgres.public.subway_info')
2025-07-01 02:49:47,327 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')
2025-07-01 02:49:47,4

2025-07-01 02:49:48,457 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'subway_info', None, 100000)
2025-07-01 02:49:48,458 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'postgres.public.subway_info')
2025-07-01 02:49:48,458 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')
2025-07-01 02:49:48,624 - INFO - EltManager:375 - ETL/IC Done : [ mart postgres _ subway_info / True (796, 796, 0) / 4.87 ]
2025-07-01 02:49:48,625 - DEBUG - EltManager:115 - self.record is True
2025-07-01 02:49:48,708 - DEBUG - PostgreSQLTableManager:302 - ('saveTable = ', 'elt_manager_log', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')
2025-07-01 02:49:48,709 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'elt_manager_log', None, 100000)
2025-07-01 02:49:48,710 - DEBUG - PostgreSQLTableMana

In [12]:
spark.stop()

25/07/01 02:52:42 WARN ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed.
